# Otázka 1: U jakého produktu je nejlepší poměr prokliků ku zobrazení?


In [1]:
import pandas as pd
#nacteni dat, stplce uz boli upravene v konvertovanom Exceli
df_clicks= pd.read_excel("~/clicks2.xlsx",  header = 0)
df_imp = pd.read_excel("~/impressions2.xlsx",header = 0)


In [2]:
#v productId sa nachadzaju prazdne hodnoty, tak ich odstranim
df_clicks_clean=df_clicks[df_clicks["productId"] != "\\N"]
df_imp_clean=df_imp[df_imp["productId"] != "\\N" ]

In [4]:
#skontrolujem nulove hodnoty
null_click=df_clicks.isnull().sum()
null_imp =df_imp.isnull().sum()
print(null_click, "\n")

print(null_imp)


userId          0
time            0
productId       0
availability    0
pos             0
proklik         0
                0
dtype: int64 

userId             0
time               0
productId          0
category_name    438
offers             0
zobrazeni          0
Unnamed: 6         0
dtype: int64


In [5]:
#ratam pomer proklikov/zobrazeni, nezalezi, jestli mergujem na zaklade productId nebo userId, vzdy ma odkazuje na ten samy produkt

#po predchadzujec chybe kovnertujem na string
df_clicks_clean["userId"] = df_clicks_clean["userId"].astype(str)
df_imp_clean["userId"] = df_imp_clean["userId"].astype(str)

#mergujem
merged_df = pd.merge(df_clicks_clean, df_imp_clean, on="userId", suffixes=("_click", "_imp"))

#groupujem
grouped_df = merged_df.groupby("userId").agg({"proklik": "sum", "zobrazeni": "sum"}).reset_index()

#pocitam pomer
grouped_df["ratio"] = grouped_df["proklik"] / grouped_df["zobrazeni"]

# najvacsi pomer
best_ratio_product = grouped_df[grouped_df["ratio"] == grouped_df["ratio"].max()]
print(best_ratio_product)

/var/folders/6z/ncy_wsd14d118qw59ml4rfb00000gn/T/ipykernel_64420/3975248715.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clicks_clean["userId"] = df_clicks_clean["userId"].astype(str)


                   userId  proklik  zobrazeni  ratio
4674  6390190443848450048       12          3    4.0


# Otazka 2 Která produktová kategorie je nejproklikávanější a která nejzobrazovanější?

In [7]:
#u zobrazeni je to jednoduche, pretoze sa stplce nachadzaju v jednej tabulke
group_zobrazeni = df_imp.groupby('category_name')['zobrazeni'].sum().reset_index()

highest_impressions = group_zobrazeni[group_zobrazeni['zobrazeni'] == group_zobrazeni['zobrazeni'].max()]

print(highest_impressions)

        category_name  zobrazeni
870  Mobilní telefony       1852


In [9]:
#u proklikov je potreba zmergovat tabulky, mergujem na zaklade userId, vysledok koreluje s vysledkom poctu zobrazeni u kategorie mobilnich telefonu
merged_df = pd.merge(df_clicks, df_imp, on='userId', how='inner')

merged_df['commonuser'] = merged_df['userId']

output_file_path = 'merged_data3.xlsx'  
merged_df.to_excel(output_file_path, index=False)

#groupujem v novej tabulke na zaklade userId
df_merged = pd.read_excel("~/merged_data3.xlsx", header=0)
df_merged.drop_duplicates(inplace=True)

specific_category = "Mobilní telefony"  
filtered_df = df_merged[df_merged['category_name'] == specific_category]

total_proklik = filtered_df['proklik'].sum()

print(total_proklik)


1782


# Otázka 3: Má počet nabídek produktu nějaký vliv na jeho proklikovost?
    

In [13]:
df = pd.read_excel("~/merged_data3.xlsx", header=0)

column1 = 'proklik'  
column2 = 'offers' 

correlation = df[[column1, column2]].corr().iloc[0, 1]

print(f"The correlation between {column1} and {column2} is: {correlation}")

The correlation between proklik and offers is: 0.009664515082090068


# Otázka 4: Jaké další zajímavé otázky vás nad danou datovou sadou napadají?

1.Ktorá kategória má najväčší rozdiel medzi zobrazeniami a preklikmi?
2.Má pozícia vplyv na zobrazenia, prekliky?
3.V ktorú hodinu je najviac zobrazení a preklikov?
4.Má dostupnosť tovaru vplyv na zobrazenia a prekliky?
5.Ktorá kategória je najmenej/najviac dostupná?
6.Ktorá kategória má najlepšiu/najhoršiu pozíciu?
Je možné, že by sa dal urobiť nejaký ML clusterový model, kde by sa dali nájsť vzájomné vzťahy pomocou klaustrov?
